In [1]:
from datasets import load_dataset

wnut = load_dataset("wnut_17")

/Users/aswin/miniconda3/envs/mlenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|█████| 3394/3394 [00:00<00:00, 22504.07 examples/s]
Generating validation split: 100%|█| 1009/1009 [00:00<00:00, 29579.05 examples/s
Generating test split: 100%|██████| 1287/1287 [00:00<00:00, 27221.46 examples/s]


In [2]:
wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [3]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [4]:
from transformers import AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json: 100%|█████████████████| 28.0/28.0 [00:00<00:00, 65.1kB/s]
config.json: 100%|█████████████████████████████| 483/483 [00:00<00:00, 1.32MB/s]
vocab.txt: 100%|█████████████████████████████| 232k/232k [00:00<00:00, 2.08MB/s]
tokenizer.json: 100%|█████████████████████████| 466k/466k [00:00<00:00, 860kB/s]


In [6]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 '@',
 'paul',
 '##walk',
 'it',
 "'",
 's',
 'the',
 'view',
 'from',
 'where',
 'i',
 "'",
 'm',
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'empire',
 'state',
 'building',
 '=',
 'es',
 '##b',
 '.',
 'pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.',
 '[SEP]']

In [8]:
from scipy.stats import median_abs_deviation
import pandas as pd
import numpy as np

In [5]:
overall_data = pd.read_excel('tfcu_member.xlsx', index_col=0, sheet_name='KPI Data')
df_ = overall_data.drop(overall_data.index[0:3])
df = df_.drop(columns=['Unnamed: 10'])
df_time = df[['Week','Member Retention',
       'Employee Retention', 'Member Satisfaction NPS', 'Fraud rate',
       'Employee satisfaction', 'Number of Employee Relations Issues',
       '# of Member complaints', 'Member acquisition cost (CAC)',
       'Number of Compliance Issues', 'Compliance Investigation Cycle Time',
       'Decline rate', 'Financial impact due to attacks',
       'Attempted vs Fraudulent Transactions', 'Final Approval rate',
       'Fraudulent Vs Declined Transactions', 'System Uptime',
       'Severity of attacks', 'Mean Time Between Failures (MTBF)',
       'Cost per incident', 'Number of cybersecurity incidents reported'
             ]].groupby(['Week']).mean()

In [11]:
df_anomaly = pd.DataFrame(df_time['Member Satisfaction NPS'])

In [13]:
mad = median_abs_deviation(df_anomaly['Member Satisfaction NPS'])
median = np.median(df_anomaly['Member Satisfaction NPS'])
mad, median

(0.40000000000000213, 62.63125)

In [14]:
def robust_z_score(x):
    return 0.6745 * (x - median)/mad

df_anomaly['z_score'] = df_anomaly['Member Satisfaction NPS'].apply(robust_z_score)

In [16]:
df_anomaly['anomaly'] = 0
df_anomaly.loc[df_anomaly['z_score']>=3.5, 'anomaly'] = 1
df_anomaly.loc[df_anomaly['z_score']<=-3.5, 'anomaly'] = 1

In [18]:
df_anomaly[df_anomaly['anomaly']==1]

,Member Satisfaction NPS,z_score,anomaly
Week,,,
WK 20,59.99,-4.453808,1
